In [1]:
import os, cv2, glob
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def load_images_from_folder(folder, label):
    data, labels = [], []
    for f in glob.glob(os.path.join(folder, "*.png")):
        img = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (128,128))
        data.append(img.flatten())
        labels.append(label)
    return np.array(data), np.array(labels)


X_h, y_h = load_images_from_folder("/content/drive/MyDrive/spectrograms_healthy", 0)
X_10, y_10 = load_images_from_folder("/content/drive/MyDrive/spectrograms_faulty_10ohm", 3)
X_30, y_30 = load_images_from_folder("/content/drive/MyDrive/spectrograms_faulty_30ohm", 2)
X_60, y_60 = load_images_from_folder("/content/drive/MyDrive/spectrograms_faulty_60ohm", 1)


X = np.vstack((X_h, X_10, X_30, X_60))
y = np.hstack((y_h, y_10, y_30, y_60))

print(" Data shape:", X.shape, "Labels:", y.shape)


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=100, random_state=42)
X_pca = pca.fit_transform(X_scaled)

print("Reduced feature shape:", X_pca.shape)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, stratify=y, random_state=42)

svm = SVC(kernel='rbf', C=5, gamma='scale', probability=True)
svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)

print(" Training complete.")
print(classification_report(y_test, y_pred, target_names=["Healthy", "Faulty_10","Faulty_30","Faulty_60"]))


In [ ]:
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='turbo', xticklabels=["Healthy", "Faulty_10","Faulty_30","Faulty_60"], yticklabels=["Healthy", "Faulty_10","Faulty_30","Faulty_60"])
plt.title("SVM Confusion Matrix")
plt.xlabel("Predicted"); plt.ylabel("True"); plt.show()



In [ ]:
from sklearn.metrics import accuracy_score
import joblib

print("Accuracy:", accuracy_score(y_test, y_pred) * 100, "%")



In [ ]:
import numpy as np
print(np.bincount(y))


In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
# cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(svm, X_scaled, y, cv=5)
print("Cross-val accuracies:", scores)
print("Mean:", scores.mean())


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print(" Random Forest training complete!")
print(classification_report(y_test, y_pred_rf, target_names=["Healthy", "Faulty_10","Faulty_30","Faulty_60"]))
print("Accuracy:", accuracy_score(y_test, y_pred_rf)*100, "%")


In [ ]:
scores = cross_val_score(rf, X_scaled, y, cv=5)
print("Cross-val accuracies:", scores)
print("Mean:", scores.mean())
